In [ ]:
        
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
# import seaborn

import sys
# sys.path.append("/home/vegard/local_env/lib/python3.8/site-packages/")
# !pip install pyvista
import utm
import requests

#### partitioning  data int squares

In [ ]:


#image dataframe
dat = pd.read_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')

#location dataframe
dat_loc = pd.read_csv('dataset/image_data/csv/location_images_top_100_csv.csv')

#locations not in image dataframe
dat_loc = dat_loc[~dat_loc['Id'].isin(dat['Id'].to_list())]


In [ ]:
#adding target number/index to dataframe
target_dict = { j:i for i,j in enumerate(dat['Vitenskapelig navn'].unique())}

dat['target'] = [target_dict[i] for i in dat['Vitenskapelig navn'].to_list()]
dat_loc['target'] = [target_dict[i] for i in dat_loc['Vitenskapelig navn'].to_list()]

In [ ]:
max_n = dat_loc['Nordkoordinat'].max()

min_n = dat_loc['Nordkoordinat'].min()

max_e = dat_loc['Østkoordinat'].max()

min_e = dat_loc['Østkoordinat'].min()

print(max_n,min_n)
print(max_e,min_e)



In [ ]:
max_n = 8767538 
min_n = 6448621
max_e = 1108075  
min_e = -75864


def get_rectangle(point, total_squares=20):
    N = np.linspace(min_n-1, max_n, num=total_squares, endpoint=True)
    E = np.linspace(min_e-1, max_e, num=total_squares, endpoint=True)
    square_n = bisect.bisect_left(N, point[1])
    square_e = bisect.bisect_left(E, point[0])
    return [square_n,square_e]

def get_squares(point, total_squares_east=20):
    E = np.linspace(min_e-1, max_e, num=total_squares_east, endpoint=True)
    N_step = round(E[1]-E[0])
    N = np.arange(min_n-1,max_n+N_step, N_step)
    square_n = bisect.bisect_left(N, point[1])
    square_e = bisect.bisect_left(E, point[0])
    return [square_n,square_e]



In [ ]:

dat['points'] = [[p[0],p[1]] for p in zip(dat['Østkoordinat'].to_list(),dat['Nordkoordinat'].to_list())]



In [ ]:
dat['squares_8'] = [get_squares(p,8) for p in dat['points'].to_list()]
dat['squares_20'] = [get_squares(p,20) for p in dat['points'].to_list()]
dat['squares_15'] = [get_squares(p,15) for p in dat['points'].to_list()]
dat['squares_10'] = [get_squares(p,10) for p in dat['points'].to_list()]
dat['squares_30'] = [get_squares(p,30) for p in dat['points'].to_list()]
dat['squares_40'] = [get_squares(p,40) for p in dat['points'].to_list()]
dat['squares_50'] = [get_squares(p,50) for p in dat['points'].to_list()]
dat['squares_60'] = [get_squares(p,60) for p in dat['points'].to_list()]
dat['squares_70'] = [get_squares(p,70) for p in dat['points'].to_list()]
dat['squares_80'] = [get_squares(p,80) for p in dat['points'].to_list()]
dat['squares_90'] = [get_squares(p,90) for p in dat['points'].to_list()]
dat['squares_100'] = [get_squares(p,100) for p in dat['points'].to_list()]
dat['squares_150'] = [get_squares(p,100) for p in dat['points'].to_list()]
dat['squares_200'] = [get_squares(p,100) for p in dat['points'].to_list()]

dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')

In [ ]:


dat_loc['squares_8'] = [get_squares(p,8) for p in dat_loc['points'].to_list()]
dat_loc['squares_20'] = [get_squares(p,20) for p in dat_loc['points'].to_list()]
dat_loc['squares_15'] = [get_squares(p,15) for p in dat_loc['points'].to_list()]
dat_loc['squares_10'] = [get_squares(p,10) for p in dat_loc['points'].to_list()]
dat_loc['squares_30'] = [get_squares(p,30) for p in dat_loc['points'].to_list()]
dat_loc['squares_40'] = [get_squares(p,40) for p in dat_loc['points'].to_list()]
dat_loc['squares_50'] = [get_squares(p,50) for p in dat_loc['points'].to_list()]
dat_loc['squares_60'] = [get_squares(p,60) for p in dat_loc['points'].to_list()]
dat_loc['squares_70'] = [get_squares(p,70) for p in dat_loc['points'].to_list()]
dat_loc['squares_80'] = [get_squares(p,80) for p in dat_loc['points'].to_list()]
dat_loc['squares_90'] = [get_squares(p,90) for p in dat_loc['points'].to_list()]
dat_loc['squares_100'] = [get_squares(p,100) for p in dat_loc['points'].to_list()]
dat_loc['squares_150'] = [get_squares(p,100) for p in dat_loc['points'].to_list()]
dat_loc['squares_200'] = [get_squares(p,100) for p in dat_loc['points'].to_list()]


dat_loc.to_csv('dataset/image_data/csv/location_images_top_100_csv_2.csv')

## Creating relative frequency dataset.

In [ ]:
frequency_dict = {}

plant_names = list(dat_loc['target'].value_counts().index)
plant_count = list(dat_loc['target'].value_counts())

for name,count in zip(plant_names,plant_count):
    frequency_dict[name] = {}
    frequency_dict[name]['total_count'] = count
    
frequency_dict

In [ ]:
from tqdm import tqdm
import numpy as np 

frequency_dict
targets = dat_loc['target'].unique()
# frequency_in_square = {}

sq = ['squares_8','squares_20','squares_15', 'squares_10', 'squares_30', 'squares_40', 'squares_50','squares_60', 'squares_70', 'squares_80', 'squares_90', 'squares_100','squares_150', 'squares_200']
for s in tqdm(sq,position =0,leave=True):
    square_dict = {}
    for square in tqdm(dat_loc[s].unique(),position = 1, leave=False):
        temp_dat = dat_loc[dat_loc[s]==square]
        temp_freq = np.zeros(100)
        for target in targets:
            temp_freq[target] = temp_dat[temp_dat.target==target].shape[0]/frequency_dict[target]['total_count']
        square_dict[square] = list(temp_freq)
        if sorted(set(square_dict[square]))[0]==0:
            second_smallest = sorted(set(square_dict[square]))[1]
            for i,j in enumerate(square_dict[square]):
                if j==0:
                    square_dict[square][i]=second_smallest



    dat_square = pd.DataFrame(square_dict)
    dat_square.to_csv(f'{s}_relative_freq_min.csv')
    
for s in tqdm(sq,position =0,leave=True):
    square_dict = {}
    for square in tqdm(dat_loc[s].unique(),position = 1, leave=False):
        temp_dat = dat_loc[dat_loc[s]==square]
        temp_freq = np.zeros(100)
        for target in targets:
            temp_freq[target] = temp_dat[temp_dat.target==target].shape[0]/frequency_dict[target]['total_count']
        square_dict[square] = list(temp_freq)
        # if sorted(set(square_dict[square]))[0]==0:
        #     second_smallest = sorted(set(square_dict[square]))[1]
        #     for i,j in enumerate(square_dict[square]):
        #         if j==0:
        #             square_dict[square][i]=second_smallest



    dat_square = pd.DataFrame(square_dict)
    dat_square.to_csv(f'{s}_relative_freq.csv')

### adding in area values to dataframe

In [ ]:
points = dat_loc[['Østkoordinat','Nordkoordinat']].to_numpy()




point_tree = spatial.cKDTree(points)


def get__points_within_targets(point, distance=1000):

    return_list = point_tree.query_ball_point([[int(point[0]),int(point[1])]], distance)[0]
    return_dat = dat_loc.iloc[return_list]
    
    # return_list.remove(id)
    
    
    return return_dat['target']



In [ ]:
in_100=[]
in_200=[]
in_500=[]
in_1000=[]
in_1500=[]
for i in tqdm(range(dat.shape[0])):
    # print(dat.iloc[i])
    points_in_area = get_points_within(dat.iloc[i],distance = 100)
    
    p_i_a_l = list(points_in_area['target'])
    z = np.zeros(100)
    z[p_i_a_l] = 1
    in_100.append(list(z))
dat['in_100']=in_100

for i in tqdm(range(dat.shape[0])):
    # print(dat.iloc[i])
    points_in_area = get_points_within(dat.iloc[i],distance = 200)
    
    p_i_a_l = list(points_in_area['target'])
    z = np.zeros(100)
    z[p_i_a_l] = 1
    in_200.append(list(z))
dat['in_200']=in_200

for i in tqdm(range(dat.shape[0])):
    # print(dat.iloc[i])
    points_in_area = get_points_within(dat.iloc[i],distance = 500)
    
    p_i_a_l = list(points_in_area['target'])
    z = np.zeros(100)
    z[p_i_a_l] = 1
    in_500.append(list(z))
dat['in_500']=in_500


for i in tqdm(range(dat.shape[0])):
    # print(dat.iloc[i])
    points_in_area = get_points_within(dat.iloc[i],distance = 1000)
    
    p_i_a_l = list(points_in_area['target'])
    z = np.zeros(100)
    z[p_i_a_l] = 1
    in_1000.append(list(z))
dat['in_1000']=in_1000


for i in tqdm(range(dat.shape[0])):
    # print(dat.iloc[i])
    points_in_area = get_points_within(dat.iloc[i],distance = 1500)
    
    p_i_a_l = list(points_in_area['target'])
    z = np.zeros(100)
    z[p_i_a_l] = 1
    in_1500.append(list(z))
dat['in_1500']=in_1500

dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')

##  Adding height info

In [ ]:
import pandas as pd
# dat = pd.read_csv('dataset/lokasjon_top_185_csv.csv')

# dat = pd.read_csv('dataset/top185_in_oslo_area.csv')
# dat = dat[['Id','Østkoordinat','Nordkoordinat','Vitenskapelig navn']]

dat = pd.read_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')


In [ ]:
E = dat['Østkoordinat']
N = dat['Nordkoordinat']





len(E)//50

indexes = [(i*50,(i+1)*50) for i in range(len(E)//50)]
    
indexes.append((indexes[-1][1],len(E)))
height = []
source =[]

def get_index_height_list(index_slice):
    i=index_slice
    coord_string = []
    for j,k in zip(E[i[0]:i[1]],N[i[0]:i[1]]):
        coord_string.append([j,k])

        query ={
          "koordsys": 32633,
          "punkter": str(coord_string)
        }
    r = requests.get("https://ws.geonorge.no/hoydedata/v1/datakilder/dom1/punkt",params= query)
    # r = requests.get("https://ws.geonorge.no/hoydedata/v1/hoydekurver/dom1/punkt",params= query)
    list_index_height = []
    if r.status_code ==200:
        
        for i,j in enumerate(range(i[0],i[1])):
            list_index_height.append((j,r.json()['punkter'][i]['z']))

    else:
        list_index_height = [(i,None) for i in range(i[0],i[1])]
    return list_index_height

def get_index_height_list_m(index_slice):
    indexes=[i[0] for i in index_slice]
    E = [i[1] for i in index_slice]
    N = [i[2] for i in index_slice]
    coord_string = []
    for j,k in zip(E,N):
        coord_string.append([j,k])

        query ={
          "koordsys": 32633,
          "punkter": str(coord_string)
        }
    r = requests.get("https://ws.geonorge.no/hoydedata/v1/hoydekurver/dom1/punkt",params= query)
    list_index_height = []
    if r.status_code ==200:
        
        for i,j in enumerate(indexes):
            list_index_height.append((j,r.json()['punkter'][i]['z']))

    else:
        list_index_height = [(i,None) for i in indexes]
    return list_index_height



def interpol_height(index_slice):
    index_slice
    coord_string = []
    
    for i in index_slice:
        coord_string.append([E[i[0]],N[i[0]]])
        
        
    # coord_string.append([j,k])

    query ={
      "koordsys": 32633,
      "punkter": str(coord_string)
    }
    r = requests.get("https://ws.geonorge.no/hoydedata/v1/datakilder/dom1/punkt",params= query)
    list_index_height = []
    if r.status_code ==200:
        
        for i,j in enumerate(index_slice):
            list_index_height.append((j[0],r.json()['punkter'][i]['z']))

    else:
        list_index_height = [(j[0],None) for j in index_slice]
    return list_index_height



In [ ]:
from concurrent import futures
import requests

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


height_index_list = []
for group_indexes in tqdm(chunker(indexes, 10),total=len(indexes)/10):
    with futures.ThreadPoolExecutor() as executor:
        height_index_list+=(list(executor.map(get_index_height_list, group_indexes)))


In [ ]:
height_index_dict = {}

In [ ]:
height_index_list
height_index_list1 = []
# 
for i in height_index_list:
    height_index_list1+=i
# height_index_dict 
# height_index_list1
for i in height_index_list1:
    height_index_dict[i[0]]=i[1]

#### Repeat from here

In [ ]:
def get_index_height_list(index_slice):
    i=index_slice
    coord_string = []
    for j,k in zip(E[i[0]:i[1]],N[i[0]:i[1]]):
        coord_string.append([j,k])

        query ={
          "koordsys": 32633,
          "punkter": str(coord_string)
        }
    r = requests.get("https://ws.geonorge.no/hoydedata/v1/datakilder/dom1/punkt",params= query)
    # r = requests.get("https://ws.geonorge.no/hoydedata/v1/hoydekurver/dom1/punkt",params= query)
    list_index_height = []
    if r.status_code ==200:
        
        for i,j in enumerate(range(i[0],i[1])):
            list_index_height.append((j,r.json()['punkter'][i]['z']))

    else:
        list_index_height = [(i,None) for i in range(i[0],i[1])]
    return list_index_height

In [ ]:
height_index_list
height_index_list1 = []
# 
for i in height_index_list:
    height_index_list1+=i
# height_index_dict 
# height_index_list1
for i in height_index_list1:
    height_index_dict[i[0]]=i[1]

In [ ]:

empty_index = []

for k in height_index_dict:
    if not height_index_dict[k]:
        # print(k,height_index_dict[k])
        empty_index.append(k)

    
print('Numeber of missing heights:',len(empty_index))

In [ ]:
Em=[]
Nm = []
for i in empty_index:
    Em.append(E[i])
    Nm.append(N[i])
    
empty_index_Em_Nm = [i for i in zip(empty_index,Em,Nm)]

In [ ]:
v = []

for group_indexes in chunker(empty_index_Em_Nm, 50):
    v += get_index_height_list_m(group_indexes)


In [ ]:
for i in v:
    height_index_dict[i[0]]=i[1]
    
list_heights = [height_index_dict[i] for i in height_index_dict]

In [ ]:
dat['height']=list_heights
len(list_heights)

In [ ]:
len(Em)//50

indexes = [(i*50,(i+1)*50) for i in range(len(Em)//50)]
    
indexes.append((indexes[-1][1],len(Em)))

height_index_list = []
for group_indexes in tqdm(chunker(indexes, 10),total=len(indexes)/10):
    with futures.ThreadPoolExecutor() as executor:
        height_index_list+=(list(executor.map(get_index_height_list, group_indexes)))


#### To here until len(empty_index) is as close to the length of the dataset as possible

In [ ]:
#fill in missing as mean
dat['height'] = dat['height'].fillna(dat['height'].mean())
#or fill as 0 

# dat['height'] = dat['height'].fillna(0)

In [ ]:


dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')

In [ ]:
for i in tqdm(indexes):
    coord_string = []
    for j,k in zip(Em[i[0]:i[1]],Nm[i[0]:i[1]]):
        coord_string.append([j,k])
        
    query ={
      "koordsys": 32633,
      "punkter": str(coord_string)
    }
#     try:
    r = requests.get("https://ws.geonorge.no/hoydedata/v1/datakilder/dom1/punkt",params= query)
#     except:
#         height.extend([None]*50)
    
    if r.status_code ==200:
#         print(r.json())
        for p in r.json()['punkter']:
            heightm.append(p['z'])
    else:
        heightm.extend([None]*50)


In [ ]:
ENm =[(i,j) for i,j in zip(Em,Nm)]

count=0
for i,j in enumerate(height):
    if j== None and (E[i],N[i]) in ENm :
        index = ENm.index((E[i],N[i]))
        height[i]= heightm[index]
        if E[i] == 260566 and N[i]==7031548:
            height[i] = 0
        source[i] = 'dom1'
        count+=1


In [ ]:
dat['height']=height

# dat.to_csv('dataset/top185_in_oslo_area.csv')
# dat.to_csv('dataset/top185_in_oslo_area.csv')

# dat.to_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117.csv')
dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')

In [ ]:
from osgeo import gdal

In [3]:
import pandas as pd


## Get AR-50 data

In [4]:
# READ ar50 data. found at https://kart8.nibio.no/nedlasting/dashboard
#converted to parquet data.

#iterating throughj every polygon for every point.
#

import fiona 
import geopandas as gpd

gdf = gpd.read_parquet('C:/Users/vjosv/master/dataset/images/csv/ar50_data.parquet')
gdf.head()

,artype,arskogbon,artreslag,arjordbr,arveget,areal,arkartstd,kilde,geometry
0,50,98,39,98,99,1.188990e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,"MULTIPOLYGON (((-40290.000 6524970.000, -40290..."
1,99,98,98,98,98,6.961873e+09,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,"MULTIPOLYGON (((0.000 6980300.000, 0.000 69802..."
2,82,98,98,98,98,2.572468e+09,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,"MULTIPOLYGON (((-20480.000 6901090.000, -20487..."
3,50,98,39,98,55,1.165619e+05,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,"MULTIPOLYGON (((-39570.000 6877420.000, -39570..."
4,50,98,39,98,51,2.457512e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,"MULTIPOLYGON (((-39930.000 6880690.000, -39930..."


In [5]:
import pandas as pd


# dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117.csv')

# dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2.csv')

# dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images20_see1207_validation.csv')

dat = pd.read_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')

dat.head()
points = dat[['Østkoordinat','Nordkoordinat']].to_numpy()
ids_ = list(dat['Id'])

points = [(i,p) for i,p in zip(ids_,points)]

# ids = [i for i in range(len(points))]

print(len(points))



2500


In [6]:
from tqdm import tqdm
from shapely import geometry

dat.columns

artype = []
arskogbon=[]
artreslag = []
arjordbr = []
arveget = []
areal = []
arkartstd = [] 
kilde = []

list_geometry = list(gdf['geometry'])

def get_geo_info(point):
    id_ = point[0]
    p = geometry.Point(point[1])
    for i,geo in enumerate(list_geometry):
        if geo.contains(p):
            return [id_,
            gdf['artype'].iloc[i],
            gdf['arskogbon'].iloc[i],
            gdf['artreslag'].iloc[i],
            gdf['arjordbr'].iloc[i],
            gdf['arveget'].iloc[i],
            gdf['arkartstd'].iloc[i],
            gdf['kilde'].iloc[i]]


# This file provides a long_running decorator to indicate that a function needs a long amount of time to complete and
# the computer should not enter standby. This file currently only works on Windows and is a no-op on other platforms.

#copied from Kevin Barnes/kbarnes3: https://gist.github.com/kbarnes3/3fb7d353e9bdd3efccd5

import ctypes
import platform

ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001


def _set_thread_execution(state):
    ctypes.windll.kernel32.SetThreadExecutionState(state)


def prevent_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)


def allow_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS)


def long_running(func):
    def inner(*args, **kwargs):
        prevent_standby()
        result = func(*args, **kwargs)
        allow_standby()
        return result
    return inner

In [7]:
from concurrent import futures
   

def save_to_csv(list_):
    csv_dict = {}
    artype = []
    arskogbon=[]
    artreslag = []
    arjordbr = []
    arveget = []
    areal = []
    arkartstd = [] 
    kilde = []
    index = []
    for l in list_:
        for i in l:
            if i != None:
                index.append(i[0])
                artype.append(i[1])
                arskogbon.append(i[2])
                artreslag.append(i[3])
                arjordbr.append(i[4])
                arveget.append(i[5])
                arkartstd.append(i[6])
                kilde.append(i[7])
    csv_dict['index']= index
    csv_dict['artype']= artype
    csv_dict['arskogbon']= arskogbon
    csv_dict['artreslag']=artreslag
    csv_dict['arjordbr']=arjordbr
    csv_dict['arveget']=arveget
    csv_dict['arkartstd']=arkartstd
    csv_dict['kilde']=kilde
    
    csv_df = pd.DataFrame(csv_dict)
    csv_df.to_csv(f'dataset/image_data/csv/areal_info_{index[0]}_{index[-1]}.csv')    

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))



area_info_list = []
# counter = 0

# for group_points
@long_running
def concurrent_get_location_info(points):
    global area_info_list
    # area_info_list = []
    counter = 0
    for group_points in tqdm(chunker(points, 4),total=len(points)/4):

        with futures.ThreadPoolExecutor() as executor:
            area_info_list.append(list(executor.map(get_geo_info, group_points)))

        counter +=1

        if counter%100==0:
            save_to_csv(area_info_list)
            area_info_list = []
            
# concurrent_get_location_info(points[:10000])
            




In [8]:
concurrent_get_location_info(points)


save_to_csv(area_info_list)

100%|██████████| 500/500.0 [3:45:39<00:00, 27.08s/it]  


In [5]:
a='areal_info_'
a[0:10]

'areal_info'

#### Combining csv from AR 50 data

In [ ]:
file_list = [f for f in os.listdir('dataset/image_data/csv/') if f[0:10]=='areal_info']

ar_dat = pd.read_csv(file_list[0])

for f in file_list[1:]:
    
    
    temp_dat = pd.read_csv(f)
    
#     temp_dat = temp_dat.drop(columns=['Prosjekt', 'Bestemmelsesdato','Samlingsnummer','Bekrefter','Artsbestemt av','Samlingsbeskrivelse','Skjermet funn'])
    # temp_dat = temp_dat[['Id','Østkoordinat','Nordkoordinat','Vitenskapelig navn','Originale koordinater','Nøyaktighet']]

    ar_dat = pd.concat([ar_dat,temp_dat], ignore_index = True)


ar_dat.to_csv('dataset/image_data/csv/combined_areal_info_AR50.csv')
        

#### merging ar50 data and combined imag data

In [10]:

import pandas as pd
df = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_areal_info_AR50.csv')

dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv')
dat.columns

df = df.rename(columns={"index": "Id"})
dat2 = dat.merge(df, on='Id', how='left')


100%|██████████| 125/125.0 [55:46<00:00, 26.77s/it] 


## ADDING Landscape data to the combined dataframe

In [12]:
# dat_img['landscape'] = np.nan#dat_img.shape[0]*['None']
# dat_img['landscape_subtype'] = np.nan#dat_img.shape[0]*['None']
# dat_img
# dat_img['landscape'] =[0]*dat_img.shape[0]


100%|█████████▉| 585/585.75 [5:33:11<00:22, 29.81s/it]   C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\tqdm\std.py:639: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 586/585.75 [5:33:18<-1:59:52, 34.13s/it]


In [14]:
dat = pd.read_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')
dat

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Id,Taksonsorteringsrekkefølge,Valideringsstatus,Rødlistekategori,Artsnavn,Vitenskapelig navn,...,square_40x40,square_50x50,square_60x60,square_70x70,square_80x80,square_90x90,square_100x100,square_150x150,square_200x200,height
0,0,34,34,57,33385569,511000,Ikke validert (funnet er ikke kvalitetssikret),NaN,landøyde,Jacobaea vulgaris,...,"[4, 11]","[4, 14]","[5, 17]","[6, 20]","[7, 22]","[8, 25]","[8, 28]","[8, 28]","[8, 28]",85.940000
1,1,45,45,84,33378441,5690000,Ikke validert (funnet er ikke kvalitetssikret),NaN,furu,Pinus sylvestris,...,"[7, 14]","[8, 18]","[10, 21]","[12, 24]","[13, 28]","[15, 31]","[16, 35]","[16, 35]","[16, 35]",643.270000
2,2,75,75,186,33362995,1699000,Ikke validert (funnet er ikke kvalitetssikret),SE,parkslirekne,Reynoutria japonica,...,"[5, 2]","[7, 2]","[8, 3]","[9, 3]","[10, 3]","[12, 4]","[13, 4]","[13, 4]","[13, 4]",115.200000
3,3,88,88,212,33359035,4666000,Ikke validert (funnet er ikke kvalitetssikret),NaN,blekmarihand,Dactylorhiza maculata,...,"[23, 27]","[29, 34]","[35, 41]","[41, 48]","[47, 55]","[53, 62]","[59, 69]","[59, 69]","[59, 69]",110.050000
4,4,105,105,249,33351321,1922000,Ikke validert (funnet er ikke kvalitetssikret),SE,kjempespringfrø,Impatiens glandulifera,...,"[4, 10]","[4, 12]","[5, 14]","[6, 16]","[7, 19]","[7, 21]","[8, 23]","[8, 23]","[8, 23]",211.402159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,4838,33939,55443,183413,17512970,4227000,Ikke validert (funnet er ikke kvalitetssikret),NaN,bergfrue,Saxifraga cotyledon,...,"[18, 19]","[22, 24]","[27, 28]","[31, 33]","[36, 38]","[40, 43]","[44, 47]","[44, 47]","[44, 47]",211.402159
4839,4839,33943,55715,184258,17443637,4227000,Ikke validert (funnet er ikke kvalitetssikret),NaN,bergfrue,Saxifraga cotyledon,...,"[18, 19]","[22, 24]","[27, 28]","[31, 33]","[36, 38]","[40, 43]","[44, 47]","[44, 47]","[44, 47]",211.402159
4840,4840,33947,55891,184955,17358518,4227000,Ikke validert (funnet er ikke kvalitetssikret),NaN,bergfrue,Saxifraga cotyledon,...,"[5, 3]","[6, 4]","[8, 5]","[9, 5]","[10, 6]","[11, 7]","[12, 7]","[12, 7]","[12, 7]",272.930000
4841,4841,33953,56867,188058,16829202,4227000,Ikke validert (funnet er ikke kvalitetssikret),NaN,bergfrue,Saxifraga cotyledon,...,"[3, 6]","[4, 8]","[5, 10]","[5, 11]","[6, 13]","[7, 14]","[8, 16]","[8, 16]","[8, 16]",784.790000


In [ ]:
import ast
import geojson
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import pandas as pd
import numpy as np
import shapely
import geopandas as gpd




# file_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images20_see1207_validation.csv'

file_path = 'dataset/image_data/csv/combined_location_top_100_images_csv.csv'
dat = pd.read_csv('dataset/image_data/csv/combined_location_top_100_images_csv.csv')
# file_name = 'combined_lokasjon_top_100_images20_see1207_validation.csv'
file_name = 'combined_location_top_100_images_csv.csv'

dat_img = pd.read_csv(file_path)




# dat_img =dat1.merge(dat2[['Id','landscape_subtype','landscape']], on='Id', how='left')
dat_img['points']= [[i,j] for i, j in zip(dat_img.Østkoordinat, dat_img.Nordkoordinat)]

geo_df = gpd.read_file('dataset/landskaps_indeling/Marine landskap.geojson')
geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])
id_points = [(i[0],i[1],i[2]) for i in zip(list(dat_img['Id']),list(dat_img['points']),list(dat_img['landscape']))]

points = []
ids_ = []
landscape = dat_img['landscape'].to_list()
landscape_subtype = dat_img['landscape_subtype'].to_list()


for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(geo_list):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Marine_landskap'
                    landscape_subtype[j]=geo_name_list[k]
                    
            except shapely.errors.GEOSException:
                pass
            
dat_img['landscape'] = landscape
dat_img['landscape_subtype'] = landscape_subtype
id_points = [(i[0],i[1],i[2]) for i in zip(list(dat_img['Id']),list(dat_img['points']),list(dat_img['landscape']))]

dat_img.to_csv(file_name)
# pd_dict={}
# pd_dict['id'] = ids_
# pd_dict['landscape_subtype'] = landscape_subtype
# pd_dict['landscape'] = landscape
# pd_dict['points'] = points

# dataframe = pd.DataFrame(pd_dict)
# dataframe.to_csv('landscape_df_all.csv')

geo_df = gpd.read_file('dataset/landskaps_indeling/Innlandsås- og fjellandskap.geojson')

geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])


for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(geo_list):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Innlandsås- og fjellandskap'
                    landscape_subtype[j]=geo_name_list[k]
            except shapely.errors.GEOSException:
                pass

dat_img['landscape'] = landscape
dat_img['landscape_subtype'] = landscape_subtype
id_points = [(i[0],i[1],i[2]) for i in zip(list(dat_img['Id']),list(dat_img['points']),list(dat_img['landscape']))]
            
dat_img.to_csv(file_name)

geo_df = gpd.read_file('dataset/landskaps_indeling/Innlandsdallandskap.geojson')

geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])


for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(geo_list):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Innlandsdallandskap'
                    landscape_subtype[j]=geo_name_list[k]
            except shapely.errors.GEOSException:
                pass

dat_img.to_csv(file_name)

dat_img['landscape'] = landscape
dat_img['landscape_subtype'] = landscape_subtype
id_points = [(i[0],i[1],i[2]) for i in zip(list(dat_img['Id']),list(dat_img['points']),list(dat_img['landscape']))]



geo_df = gpd.read_file('dataset/landskaps_indeling/Innlandsslettelandskap.geojson')

geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])

for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(tqdm(geo_list)):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Innlandsslettelandskap'
                    landscape_subtype[j]=geo_name_list[k]
            except shapely.errors.GEOSException:
                pass

dat_img.to_csv(file_name)

dat_img['landscape'] = landscape
dat_img['landscape_subtype'] = landscape_subtype
id_points = [(i[0],i[1],i[2]) for i in zip(list(dat_img['Id']),list(dat_img['points']),list(dat_img['landscape']))]


geo_df = gpd.read_file('dataset/landskaps_indeling/Kystslettelandskap.geojson')

geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])

for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(geo_list):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Kystslettelandskap'
                    landscape_subtype[j]=geo_name_list[k]
            except shapely.errors.GEOSException:
                pass


dat_img.to_csv(file_name)

geo_df = gpd.read_file('dataset/landskaps_indeling//Kystås- og kystfjellandskap.geojson')

geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])

for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(geo_list):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Kystås- og kystfjellandskap'
                    landscape_subtype[j]=geo_name_list[k]
            except shapely.errors.GEOSException:
                pass


dat_img.to_csv(file_name)

dat_img['landscape'] = landscape
dat_img['landscape_subtype'] = landscape_subtype
id_points = [(i[0],i[1],i[2]) for i in zip(list(dat_img['Id']),list(dat_img['points']),list(dat_img['landscape']))]


geo_df = gpd.read_file('dataset/landskaps_indeling/Fjordlandskap.geojson')

geo_list = list(geo_df['geometry'])
geo_name_list =  list(geo_df['name'])

for j,i in enumerate(tqdm(id_points)):
    if isinstance(i[2], str):
        continue
    else:
        point= i[1]
        p = Point(point[0], point[1])
        for k,pol in enumerate(geo_list):
            try:
                
                if pol.contains(p):
                    points.append(p)
                    ids_.append(i[0])
                    landscape[j]='Fjordlandskap'
                    landscape_subtype[j]=geo_name_list[k]
            except shapely.errors.GEOSException:
                pass



dat_img['landscape']= landscape
dat_img['landscape_subtype']= landscape_subtype
dat_img.to_csv(file_name)

100%|██████████| 1765/1765 [00:00<00:00, 255635.43it/s]

100%|██████████| 1765/1765 [00:00<00:00, 96343.61it/s]

100%|██████████| 1765/1765 [00:00<00:00, 157939.63it/s]

100%|██████████| 1765/1765 [00:00<00:00, 53303.86it/s]

100%|██████████| 1765/1765 [00:00<00:00, 110185.85it/s]

100%|██████████| 1765/1765 [00:00<00:00, 91574.16it/s]

100%|██████████| 1765/1765 [00:00<00:00, 112801.65it/s]

100%|██████████| 1765/1765 [00:00<00:00, 109705.79it/s]

100%|██████████| 1765/1765 [00:00<00:00, 112853.24it/s]

100%|██████████| 1765/1765 [00:00<00:00, 151420.47it/s]

100%|██████████| 1765/1765 [00:00<00:00, 101159.41it/s]

100%|██████████| 1765/1765 [00:00<00:00, 94320.67it/s]

100%|██████████| 1765/1765 [00:00<00:00, 115654.78it/s]

100%|██████████| 1765/1765 [00:00<00:00, 118084.39it/s]

100%|██████████| 1765/1765 [00:00<00:00, 99511.33it/s]

100%|██████████| 1765/1765 [00:00<00:00, 62724.08it/s]

100%|██████████| 1765/1765 [00:00<00:00, 110221.94it/s]

100%|██████████| 1765/1765 [00:00<00:

In [17]:
dat_img.to_csv(file_path)

## Using KNN to fill in missing landscape types


Once for 'landscape' and once for 'landscape_subtype'

In [ ]:

dat = pd.read_csv(file_path)
def get_sourounding_squares(square,resolution, dat):
    if isinstance(square, str):
        square= literal_eval(square)
    
    return_list = []
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            return_list.append(str([square[0]+i,square[1]+j]))
    
    return dat[dat[resolution].isin(return_list)]



In [ ]:
landscape_list_u = dat[~dat['landscape'].isna()]['landscape'].unique()

dat_with_l = dat[~dat['landscape'].isna()]
landscape_list = dat_with_l['landscape'].to_list()
landscape_to_index = {j:i for i,j in enumerate(landscape_list_u)}

index_to_landscape = {i:j for i,j in enumerate(landscape_list_u)}
landscape_to_index

landscape_index = [landscape_to_index[i] for i in landscape_list]
dat_with_l['landscape'] = landscape_index


dat_nan = dat[dat['landscape'].isna()]

landscape_to_index

In [ ]:

from sklearn.neighbors import KNeighborsClassifier


In [ ]:
dat_with_l


path_ = dat_with_l['landscape'].to_list()
points = [ [i,j] for i,j in zip(dat_with_l['Østkoordinat'].to_list(),dat_with_l['Nordkoordinat'].to_list())]

print(len(points))
print(len(landscape_index))

points[:10]


from sklearn.neighbors import KNeighborsClassifier



neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(points, landscape_index)


In [ ]:
predicted_landscapes = []
paths = []

for square in dat['square_30x30'].unique():
    dat_sub = get_sourounding_squares(square,'square_30x30', dat)

    dat_sub_l = dat_sub[~dat_sub['landscape'].isna()]
    dat_sub_na = dat[dat['square_30x30']==square]
    dat_sub_na = dat_sub_na[dat_sub_na['landscape'].isna()]
    
    if dat_sub_na.shape[0]==0:
        continue
    if dat_sub_l.shape[0]<5:
        continue
        
    landscape_list = dat_sub_l['landscape'].to_list()


    landscape_index = [landscape_to_index[i] for i in landscape_list]



    dat_sub_na

    
    path_ = dat_sub_na['path'].to_list()


    landscape_index = [landscape_to_index[i] for i in dat_sub_l['landscape'].to_list()]
    points = [ [i,j] for i,j in zip(dat_sub_l['Østkoordinat'].to_list(),dat_sub_l['Nordkoordinat'].to_list())]

    points_predict = [ [i,j] for i,j in zip(dat_sub_na['Østkoordinat'].to_list(),dat_sub_na['Nordkoordinat'].to_list())]

    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(points, landscape_index)



    predicted_points = list(neigh.predict(points_predict))

    predicted_landscape = [index_to_landscape[i] for i in predicted_points]

    paths+= path_
    predicted_landscapes += predicted_landscape

In [ ]:
import warnings
warnings.filterwarnings("ignore")


predicted_landscapes
paths


indexes = []
for k in paths:
    for i,j in enumerate(dat['path'].to_list()):
        if k==j:
            indexes.append(i)
indexes        


for i,j in zip(indexes,predicted_landscapes):
    dat.iloc[i]['landscape']=j
    
warnings.filterwarnings("default")

##### sublandscape

In [ ]:
landscape_list_u = dat[~dat['landscape_subtype'].isna()]['landscape_subtype'].unique()

dat_with_l = dat[~dat['landscape_subtype'].isna()]
landscape_list = dat_with_l['landscape_subtype'].to_list()
landscape_to_index = {j:i for i,j in enumerate(landscape_list_u)}

index_to_landscape = {i:j for i,j in enumerate(landscape_list_u)}
landscape_to_index

landscape_index = [landscape_to_index[i] for i in landscape_list]
dat_with_l['landscape_subtype'] = landscape_index


dat_nan = dat[dat['landscape_subtype'].isna()]

landscape_to_index

In [ ]:
dat_with_l


path_ = dat_with_l['landscape'].to_list()
# landscape_index = [landscape_to_index[i] for i in dat_with_l['landscape'].to_list()]
points = [ [i,j] for i,j in zip(dat_with_l['Østkoordinat'].to_list(),dat_with_l['Nordkoordinat'].to_list())]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier



neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(points, landscape_index)



print(neigh.predict([[302501, 6956116]]))

In [ ]:
def get_sourounding_squares(square,resolution, dat):
    if isinstance(square, str):
        square= literal_eval(square)
    
    return_list = []
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            return_list.append(str([square[0]+i,square[1]+j]))
    
    return dat[dat[resolution].isin(return_list)]

In [ ]:
dat['knn1000_20x20']=[None]*dat.shape[0]

for square in tqdm(dat['square_20'].unique()):
    break
    sourounding_squares = get_sourounding_squares(square,'square_20',dat_loc_minus)
    
    list_points =[ast.literal_eval(i) for i in sourounding_squares['points'].to_list()]
    

    if len(list_points)>1000:
        neigh = KNeighborsClassifier(n_neighbors=1000)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    elif len(list_points)>500:
        neigh = KNeighborsClassifier(n_neighbors=500)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    elif len(list_points)>250:
        neigh = KNeighborsClassifier(n_neighbors=250)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    elif len(list_points)>100:
        neigh = KNeighborsClassifier(n_neighbors=100)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    else:
        continue
    predic_square = dat[dat['square_20']==square]
    
    
    predic_square_list_points =[ast.literal_eval(i) for i in predic_square['points'].to_list()]
    
    predic_square__probabilities = neigh.predict_proba(predic_square_list_points)
    

    for i,ii in enumerate(tqdm(predic_square.index)):
        temp_array = np.zeros(100)
        temp_array2 = np.zeros(100)
        
        for j,jj in enumerate(neigh.classes_):
            temp_array[jj] = neigh.predict_proba(predic_square_list_points)[i][j]
            temp_array2[jj]=1
        dat.loc[ii,'knn1000_30x30']=str(list(temp_array))
        dat.loc[ii,'in_range_1000_30x30'] =str(list(temp_array2))
        
dat['knn500_40x40']=['']*dat.shape[0]

for square in tqdm(dat['square_20'].unique()):

    
    sourounding_squares = get_sourounding_squares(square,'square_40x40',dat_loc_minus)
    
    list_points =[ast.literal_eval(i) for i in sourounding_squares['points'].to_list()]
    

    if len(list_points)>500:
        neigh = KNeighborsClassifier(n_neighbors=500)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    elif len(list_points)>250:
        neigh = KNeighborsClassifier(n_neighbors=250)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    elif len(list_points)>100:
        neigh = KNeighborsClassifier(n_neighbors=100)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    elif len(list_points)>50:
        neigh = KNeighborsClassifier(n_neighbors=50)
        neigh.fit(list_points, sourounding_squares['target'].to_list())
    else:
        continue
    predic_square = dat[dat['square_40x40']==square]
    
    
    predic_square_list_points =[ast.literal_eval(i) for i in predic_square['points'].to_list()]
    
    predic_square__probabilities = neigh.predict_proba(predic_square_list_points)
    

    for i,ii in enumerate(predic_square.index):

        
        temp_array = neigh.predict_proba(predic_square_list_points)

            

        dat.loc[ii,'knn500_40x40']=str(list(temp_array))
        # dat.loc[ii,'in_range_500_40x40'] =str(list(temp_array2))
        

### Splitting data to train, test and validate 

In [ ]:
from sklearn.model_selection import train_test_split

train_dat, test = train_test_split(dat, test_size=1-0.64)

test_dat,validation_dat = train_test_split(test, test_size=0.5555)

train_dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv_train.csv')
test_dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv_test.csv')
validation_dat.to_csv('dataset/image_data/csv/combined_location_top_100_images_csv_validation.csv')


#### creating logistic regression model

In [ ]:




df_comb = pd.concat([test_dat,train_dat,validation_dat]) 

df_comb['height'] = df_comb['height'].fillna(df_comb['height'].mean())
df_comb['Nordkoordinat'] = df_comb['Nordkoordinat'].fillna(round(df_comb['Nordkoordinat'].mean(),0))

list_h = df_comb['height'].to_list()
list_n  = df_comb['Nordkoordinat'].to_list()
print(df_comb['Nordkoordinat'].max())
print(df_comb['Nordkoordinat'].min())
print(df_comb['Nordkoordinat'].max()-df_comb['Nordkoordinat'].min())


list_n  = df_comb['Nordkoordinat'].fillna(6787704).to_list()
print(len(list_n))
print(len(list_h))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(list_h)

h_category = []
heigt_c = [100,300,600,900,1900]

for i in list_h:
    for j,k in enumerate(heigt_c):
        if i <k:
            h_category.append(j)
            break
df_comb['height_category'] = h_category

n_category = []
north_c = [6449347 +1*(2318191/5),6449347 +2*(2318191/5),6449347 +3*(2318191/5),6449347 +4*(2318191/5),6449347+10 +5*(2318191/5)]

for i in list_n:
    for j,k in enumerate(north_c):
        a = True
        if i <k:
            n_category.append(j)
            a=False
            break
    if a:
        print(i)

df_comb['north_category'] = n_category

df_comb.columns

In [ ]:
ids = df_comb['Id']
landscape = pd.get_dummies(df_comb['landscape'],drop_first =True)
treeline  = pd.get_dummies(df_comb['treeline'],drop_first =True)
arskogbon = pd.get_dummies(df_comb['arskogbon'],drop_first =True)
artreslag = pd.get_dummies(df_comb['artreslag'],drop_first =True)
   

arjordbr = pd.get_dummies(df_comb['arjordbr'],drop_first =True)
arveget  = pd.get_dummies(df_comb['arveget'],drop_first =True)
Fylke = pd.get_dummies(df_comb['Fylke'],drop_first =True)
north_category = pd.get_dummies(df_comb['north_category'],drop_first =True)
height_category = pd.get_dummies(df_comb['height_category'],drop_first =True)
landscape_subtype = pd.get_dummies(df_comb['landscape_subtype'],drop_first=True)

df = pd.concat([ids,landscape,treeline,arskogbon,artreslag,arjordbr,arveget,Fylke,landscape_subtype,height_category,north_category],axis=1)

In [ ]:
labels = df_comb['target']
id_label = pd.DataFrame({'Id':ids,'labels':labels})


In [ ]:
id_list_train = train_dat['Id'].to_list()
id_list_test = test_dat['Id'].to_list()

df_train = df[df['Id'].isin(id_list_train)].loc[:, df.columns != 'Id']
df_test = df[df['Id'].isin(id_list_test)].loc[:, df.columns != 'Id']

labels_train = id_label[id_label['Id'].isin(id_list_train)].loc[:, id_label.columns != 'Id']
labels_test= id_label[id_label['Id'].isin(id_list_test)].loc[:, id_label.columns != 'Id']

In [ ]:
import sklearn
df = df.apply(pd.to_numeric)
labels = labels.apply(pd.to_numeric)


x_train, x_test,y_train,y_test = sklearn.model_selection.train_test_split(df,labels,test_size=0.3,random_state=1)

In [ ]:
df_train.columns = df_train.columns.astype(str)
df_test.columns = df_test.columns.astype(str)


In [ ]:
logmodel = LogisticRegression(class_weight = 'balanced')
logmodel.fit(df_train,labels_train)
import pickle



filename = 'multinomial_logisticmodel1.sav'

pickle.dump(logmodel, open(filename, 'wb'))

In [ ]:
id_list_val = validation_dat['Id'].to_list()


df1 = df[~df['Id'].isin(id_list_val)]

df0 =df1.loc[:, df.columns != 'Id']
id_labels0= id_label[['labels']]


df0.columns = df0.columns.astype(str)
logmodel = LogisticRegression(class_weight = 'balanced')
logmodel.fit(df0,id_labels0)
import pickle


# df_test.columns = df_test.columns.astype(str)

# df
# id_label
filename = 'multinomial_logisticmodel1_validate.sav'

pickle.dump(logmodel, open(filename, 'wb'))